In [1]:
import numpy as np
from modules import bayesian
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

In [2]:
#Carico i dati in data e li ordino in sdata e grid

data=np.transpose(np.loadtxt('../nlttk.out'))

sdata, grid = bayesianL.generatesorteddata(data, 500)

[0. 1. 0.] [0. 0. 1.] [1. 0. 0.]


In [3]:
#Medio i dati su tutti i punti equivalenti per simmetria cubica e 
#ritorno la lista dei nuovi dati 'simmetrizzati' in gplot e sd
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
sd=np.array([dataplot, datasigmaplot])

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


In [4]:
#Calcolo il fit bayesiano con glot e sd come test
mN, SN, y_infer, sy_infer=\
bayesianL.bayesianpol(gplot, sd, 12, 50, 0.001, gplot, ifprint= True, ifwarning= True)

parametri ottimali [ 6.80465262e-01 -1.98571732e+01  2.15639742e+01  7.24953530e+01
 -2.14242198e+01 -1.49534630e+02  1.57483485e+01  2.10096341e+01
 -4.38186467e+01  6.39491087e+01  4.10729402e+00  5.95657829e+00
 -3.08903441e+01  1.02921827e+01  2.22573257e+01  5.54342661e+00
 -2.72232622e+01 -5.07729433e+00 -1.14966359e+01  1.14662416e+01
 -1.89526653e+01  1.21705175e+01  1.19811595e+01  2.08536106e+01
 -3.79835304e-01 -1.47535791e+01 -1.09647754e+01 -2.94316926e+00
  4.78601090e+00  3.54650241e+00  1.10816548e-01]
numero di armoniche cubiche 19
valore a kmin inferito 0.594961902152483 bias 0.6804652618135889 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 2.6958890813168932e+29


In [5]:
#Calcolo il best fit su gplot e sd con 30 dati
N=20
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
bayesianL.bestfit(gplot, sd, N, gplot, ifprintbestfit= False, ifprintfinal= True)

parametri ottimali [  0.81287433 -10.071352    10.95360412  14.34138857   0.04189789
 -27.39076705   9.87181039 -15.52964289 -16.37923587 -15.34040183
  14.28701991  21.68058864  10.24106696  -3.3338389   -4.35918373
  -2.25407413]
numero di armoniche cubiche 8
valore a kmin inferito 0.663342980410345 bias 0.8128743341522942 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 4.855219110416932e+24
grado ottimale 14 grado massimo tentato 28


In [6]:
#Plotto la funzione di evidence che ho massimizzato per il best fit nella cella precedente
%matplotlib widget
plt.plot(log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
#Plotto il best fit e i dati (gplot e sd)
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [8]:
#Calcolo il best fit includendo sempre piu' punti 
predb=[]
spredb=[]
Npointsb=[]
for N in range(10, 101):
    mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
    bayesianL.bestfit(gplot, sd, N, gplot, ifprintbestfit=False, ifprintfinal=False)
    Npointsb.append(np.linalg.norm(gplot[N]))
    if (N-10)%(90//10)==0: print(int((N-10)/90*100), '%')
    predb.append(mN[0])
    spredb.append(SN[0,0])

0 %
10 %
20 %
30 %
40 %
50 %
60 %
70 %
80 %
90 %
100 %


In [ ]:
#Calcolo il model averaging su gplot  e sd con 30 punti
N=20
mr, sr, logevidence=\
bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, ifprintmodpred=False, ifprintfinal=True)

In [ ]:
#Calcolo il model averaging includendo sempre piu' punti 
preda=[]
spreda=[]
Npointsa=[]
for N in range(10, 101):
    
    mr, sr, logevidence=\
    bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, ifprintmodpred=False, ifprintfinal=False)
    Npointsa.append(np.linalg.norm(gplot[N]))
    if (N-10)%(90//10)==0: print(int((N-10)/90*100), '%')
    preda.append(mr)
    spreda.append(sr)

In [20]:
%matplotlib widget
fig, ax=plt.subplots(3,constrained_layout=True, figsize=[9,9])


ax[0].set_title('Bayesian model averaging prediction at k=0 with data in range $[: k_{max}]$')

ax[0].errorbar(np.array(Npointsa), np.array(preda), np.sqrt(np.array(spreda)), fmt='.')
ax[0].plot(np.array(Npointsa), np.ones(len(Npointsa))*0.816, label='GK result')
    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')

ax[1].errorbar(np.array(Npointsb), np.array(predb), np.sqrt(np.array(spredb)), fmt='.')
ax[1].plot(np.array(Npointsb), np.ones(len(Npointsb))*0.816, label='GK result')    
ax[1].set_xlabel('$k_{max} (nm^{-1})$')
ax[1].set_ylabel('prediction at k=0')
ax[1].legend()
ax[2].set_title('Data')

ax[2].errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.', label='3375 water molecules at 300k 30ns') 

ax[2].legend()

ax[2].set_xlabel(r'k ($nm^{-1}$)')
#plt.savefig('nlttcubic.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …